# Skulstripping Evaluation
## Intro
Given a list of nii.gz files inside the 'eval' folder, makes a prediction for the brain mask for each and saves it in the 'eval/predictions' folder unless specified otherwise.

## Libraries

In [16]:
import os
import importlib
import numpy as np
import matplotlib.pyplot as plt
from utils.vedo import plot_slicer_cloud, plot_volume_cloud

import utils.nifti
importlib.reload(utils.nifti)
from utils.nifti import estimate_volume

import importlib
import preprocessing.preprocessor
importlib.reload(preprocessing.preprocessor)
from preprocessing.preprocessor import MRIProcessor

# neural imaging
import nibabel as nib

# tensorflow
import tensorflow as tf
from evaluation.evaluation import *

# morphology
from skimage import morphology

# Make numpy printouts easier to read.
np.set_printoptions(precision=3, suppress=True)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


## Configuration
Here it's possible to change some paramaters to make predictions

Paths:
- dataset_folder (string): path to the folder that contains the files on which we want to make predictions
- save_folder (string): path to the folder in which we want to save the predictions

Save resuls:
- save (boolean): if true, saves the predictions in save_folder

3D preview:
- show_3d_preview (boolean): shows the predicted mask in an external window

Morphological smoothing:
- remove_small_objects (boolean): if true, removes small unconnected regions based on object_min_area
- object_min_area (int): the smallest allowable contiguos region size, in voxels
- fill_small_holes (boolean): if true, fills small holes
- holes_max_area (int): the maximum area, in voxels, of a contiguous hole that will be filled

Prediction parameters:
- patch_size (tuple): size of the sliding window used to extract patches from the image
- patch_resolution (tuple): desired target resolution for all patch (should be equal to the training resolution of the model)
- stride (int): translation offset of the sliding window (less is better but requires more computational time)

Suggested stride values: 6,8,12,16

In [17]:
dataset_folder = '../eval'
save_folder = '../eval/predictions'

structured_mode = False
save = True

show_3d_preview = False

remove_small_objects = False
object_min_area = 700000
fill_small_holes = False
holes_max_area = 100000

patch_size = (30,30,30) #voxels
patch_resolution = (0.2,0.2,0.2) #mm
stride = 16

pred_name = f'pred{stride}_dice_brain_mask.nii.gz'

From here the code should remain unchanged

In [18]:
def random_cropping_inference(input_volume, stride = stride, threshold = 0.5, verbose=False):
    # Make a random volume to simulate the input
    depth, height, width = input_volume.shape

    predicted_patches = np.zeros_like(input_volume)
    count_map = np.zeros_like(input_volume)

    steps_d = max(1, (depth - patch_size[0]) // stride +1)
    steps_h = max(1, (height - patch_size[1]) // stride +1)
    steps_w = max(1, (width - patch_size[2]) // stride +1)

    assert patch_size[0] <= depth
    assert patch_size[1] <= height
    assert patch_size[2] <= width

    # if steps * stride < volume size, add one more step
    if steps_d * stride < depth: steps_d += 1
    if steps_h * stride < height: steps_h += 1
    if steps_w * stride < width: steps_w += 1
    # for each patch make a prediction
    for d in range(0, steps_d * stride, stride):
        for h in range(0, steps_h* stride, stride):
            for w in range(0, steps_w * stride, stride):

                # check if the patch is out of bounds
                if d + patch_size[0] > depth:
                    d = depth - patch_size[0]
                if h + patch_size[1] > height:
                    h = height - patch_size[1]
                if w + patch_size[2] > width:
                    w = width - patch_size[2]
                
                ranges_d = (d, d+patch_size[0])
                ranges_h = (h, h+patch_size[1])
                ranges_w = (w, w+patch_size[2])

                # extract the current patch from the input volume
                patch = input_volume[ranges_d[0]:ranges_d[1], ranges_h[0]:ranges_h[1], ranges_w[0]:ranges_w[1]]
                
                # perform inference on the patch
                predicted_patch = 1

                # sum 
                predicted_patches[ranges_d[0]:ranges_d[1], ranges_h[0]:ranges_h[1], ranges_w[0]:ranges_w[1]] += predicted_patch
                count_map[ranges_d[0]:ranges_d[1], ranges_h[0]:ranges_h[1], ranges_w[0]:ranges_w[1]] += 1

    # avoid division by 1
    count_map[count_map==0] = 1
    #predicted_patches /= count_map
    #binary_predictions = (predicted_patches >= threshold).astype(np.float64)
    
    return predicted_patches

In [19]:
# visualize the predicted volume and the output of the random cropping inference simulation with vedo
import vedo
vedo.settings.default_backend= 'vtkplotter'
from vedo import *
from vedo.applications import Slicer3DPlotter

def visualize_predictions(input_volume, pred_volume, threshold = 0.5):
    plt = Plotter()

    v2 = Volume(input_volume)

    # Set voxel dimension in mm
    v2.cmap('bone')

    plt += v2

    # Define class labels for each unique value in the mask
    class_labels = np.unique(pred_volume)
    class_labels = class_labels[class_labels != 0]

    # Define a color for each unique value in the mask
    color_map = np.random.rand(len(class_labels), 3)

    # Create an empty dictionary to store point clouds
    point_clouds = {}

    i=0
    for label in class_labels:
        # Get voxel coordinates
        voxel_coords = np.array(np.where(pred_volume == label)).T  * v2.spacing()
        pts = Points(voxel_coords, r=3, c=color_map[i], alpha=0.5)
        # Multiply by voxel dimension to get coordinates in mm

        # Store the point cloud in the dictionary
        point_clouds[label] = pts
        i+=1


    for label in class_labels:
        plt += point_clouds[label]

    plt.add_slider(
        lambda w, e: [point_clouds[label].alpha(w.value) for label in class_labels],
        xmin=0,
        xmax=1,
        value=0.5,
        pos="bottom-right-vertical",
        title="Opacity",
    )

    plt.add_slider(
        lambda w, e: v2.alpha([0, w.value]),
        xmin=0,
        xmax=1,
        value=0.5,
        pos="bottom-left-vertical",
        title="Opacity",
    )

    # Make the plot rotate automatically
    plt += Text2D("Press q to exit", pos=(0.8, 0.05), s=0.8)

    return plt.show(viewup='z').close()

In [20]:
input_volume = np.random.rand(128,128,60)
pred_volume = random_cropping_inference(input_volume, stride=stride, threshold=0.5)
np.unique(pred_volume)
visualize_predictions(input_volume, pred_volume, threshold=0.5)